In [1]:
import sys
import time
import pandas as pd
import numpy as np

# QCGPU Imports
import qcgpu

# ProjectQ Imports
from projectq import MainEngine
import projectq.ops as ops
from projectq.backends import Simulator
from projectq.types import Qureg

# Qiskit Imports
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit, execute

In [11]:
def bench_qcgpu(n, depth):
    state = qcgpu.State(n)

    h = qcgpu.gate.h()
    x = qcgpu.gate.x()
    t = qcgpu.gate.t()

    print('started')
    start = time.time()

    for level in range(depth):
        for q in range(n):
    
            state.apply_gate(h, q)
            state.apply_gate(t, q)

            if q != 0:
                state.apply_controlled_gate(x, q, 0)
        
    runtime = time.time() - start
    print('ended: ', runtime)
    return runtime
    
def bench_projectq(n, depth):
    eng = MainEngine(backend=Simulator(gate_fusion=True), engine_list=[])
    qbits = eng.allocate_qureg(n)

    print('started')
    start = time.time()

    for level in range(depth):
        print(depth-level)
        for q in qbits:
            ops.H | q
            ops.T | q
            if q != qbits[0]:
                ops.CNOT | (q, qbits[0])

    runtime = time.time() - start
    print('ended: ', runtime)

    for q in qbits:
        ops.Measure | q
    eng.flush()
    return runtime

def bench_qiskit(n, depth):
    q = QuantumRegister(n)
    c = ClassicalRegister(n)
    qc = QuantumCircuit(q, c)

    
    
    for level in range(depth):
        for i in range(n):
            qc.h(q[i])
            qc.t(q[i])

            if i != 0:
                qc.cx(q[i], q[0])
        
    
    
    qc.measure(q, c)

    print('started')
    start = time.time()
    
    job_sim = execute(qc, "local_qasm_simulator")
    
    runtime = time.time() - start
    print('ended: ', runtime)
    return runtime


In [ ]:
bench_qcgpu(28,100)

In [16]:
bench_projectq(25,1)

started
1
ended:  8.024413824081421


8.024413824081421

In [17]:
bench_qiskit(25,1)

started
ended:  0.2896561622619629


0.2896561622619629